In [ ]:
UKF
https://web.statler.wvu.edu/~irl/IRL_WVU_Online_UKF_Implementation_V1.0_06_28_2013.pdf

In [1]:
import numpy as np
from scipy import linalg

In [ ]:
def sigmaUpdate(x,p,L,lam):
    '''
    Generate sigma points
    '''
    p_lower = linalg.cholesky(p)
    x1_ = x.dot(np.ones((L,1)))
    x2_ = np.sqrt(L+lam)*p_lower
    X = np.vstack((x, x1_ + x2_, x1_ - x2_))
    return X

def predictTransform(X):
    Xm = np.empty((1,L))
    for x in X:
        Xm = np.vstack((Xm,f(x)))
    xm = wm * Xm
    pm = Q;
    for i in range(2*L+1):
        pm += wc[0][i] * (Xm[i] - xm).T * (Xm(i) - xm)

def observationTransform():
    Ym = np.empty((1,M))
    for x in Xm:
        Ym = np.vstack((Ym,h(x)))
    ym = wm * Ym
    Pyy = R
    Pxy = np.zeros((L,M))
    for i in range(2*L+1):
        Pyy += wc[0][i] * (Ym[i] - ym).T * (Ym[i] - ym)
        Pxy += wc[0][i] * (Xm[i] - xm).T * (Ym[i] - ym)

def measurementUpdate():
    K = Pxy.dot(linalg.inv(Pyy))
    x = xm + (y - ym).dot(K.T)
    p = pm - K.dot(Pyy).dot(K.T)

In [45]:
class UnscentedKF:
    L = 4
    alpha = 1
    beta = 2
    kappa = 0
    lam = alpha**2 * (L+kappa) - L
    wm = np.ones((1,2*L+1)) / (2*(L+lam))
    wc = wm
    wm[0][0] = lam/(lam+L)
    wc[0][0] = lam/(lam+L) + 1 - alpha**2 + beta
    
    def __init__(self, SysFuc, ObsFunc, Qnoise, Rnoise, IniX, IniP, SysPara):
        self.L = Qnoise.shape[0]
        self.M = Rnoise.shape[0]
        self.alpha = SysPara[0]
        self.beta = SysPara[1]
        self.kappa = SysPara[2]
        self.lam = self.alpha**2 * (self.L+self.kappa) - self.L
        self.wm = np.ones((1,2*self.L+1)) / (2*(self.L+self.lam))
        self.wc = self.wm
        self.wm[0][0] = self.lam/(self.lam+self.L)
        self.wc[0][0] = self.lam/(self.lam+self.L) + 1 - self.alpha**2 + self.beta
        self.x0 = IniX
        self.p0 = IniP
        self.Q = Q
        self.R = R
        
        print('Initialization complete')
    
    def sigmaUpdate(self,x,p):
        '''
        Generate sigma points
        '''
        p_lower = linalg.cholesky(p)
        x1_ = x.dot(np.ones((self.L,1)))
        x2_ = np.sqrt(self.L+self.lam)*p_lower
        SigmaPoints = np.vstack((x, x1_ + x2_, x1_ - x2_))
        return SigmaPoints

    def UnscentedTransform(self, SigmaPoints):
        Xm = np.empty((1,self.L))
        for x in SigmaPoints:
            Xm = np.vstack((Xm,f(SigmaPoints)))
        xm = self.wm * Xm
        pm = self.Q;
        for i in range(2*self.L+1):
            pm += self.wc[0][i] * (Xm[i] - xm).T * (Xm(i) - xm)
        
        Ym = np.empty((1,self.M))
        for x in Xm:
            Ym = np.vstack((Ym,h(Xm)))
        ym = self.wm * Ym
        Pyy = self.R
        Pxy = np.zeros((self.L,self.M))
        for i in range(2*self.L+1):
            Pyy += self.wc[0][i] * (Ym[i] - ym).T * (Ym[i] - ym)
            Pxy += self.wc[0][i] * (Xm[i] - xm).T * (Ym[i] - ym)
            
        return Pyy, Pxy, ym, xm, pm

    def measurementUpdate(self,Pyy,Pxy,ym,xm,pm,y):
        K = Pxy.dot(linalg.inv(Pyy))
        x = xm + (y - ym).dot(K.T)
        p = pm - K.dot(Pyy).dot(K.T)
        return x,p
        
    def UKF_oneStep(self,x,p):
        '''
        One step UKF update
        '''
        SigmaPoints = self.sigmaUpdate(x,p)
        Pyy,Pxy,ym,xm,pm = self.UnscentedTransform(SigmaPoints)
        x,p = self.measurementUpdate(Pyy,Pxy,ym,xm,pm,y)
        
        return x,p
        

In [39]:
example = UnscentedKF(4)

he4


In [2]:
aa = np.array([1,2,3])
bb = np.array([[4,5,6]])
cc = np.array([[7,8,9],[55,55,55]])

In [22]:
dd= np.append(aa,cc,axis=0)
print(dd)

ValueError: all the input arrays must have same number of dimensions

In [23]:
ee = np.vstack((aa,cc,bb))
print(ee)

[[ 1  2  3]
 [ 7  8  9]
 [55 55 55]
 [ 4  5  6]]


In [31]:
dd = np.ones((3,1))
ee = dd.dot(bb)
print(ee)

[[ 4.  5.  6.]
 [ 4.  5.  6.]
 [ 4.  5.  6.]]


In [42]:
bb[1]

IndexError: index 1 is out of bounds for axis 0 with size 1